<a href="https://colab.research.google.com/github/amhaiskar0921/AmazonProject/blob/main/main_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pyarrow.parquet as pq
import pandas as pd
import numpy as np

# For data visualization
import seaborn as sns

Mounted at /content/drive


In [70]:
np.random.seed(42)

sample_size = 10000

shopping_data = pq.read_table('/content/drive/MyDrive/Amazon (LA) - Multi-Class Product Classification (Team A)/Datasets/shopping_queries_dataset_examples.parquet')
df = shopping_data.to_pandas().sample(n=sample_size, random_state=42)
shopping_data_p = pq.read_table('/content/drive/MyDrive/Amazon (LA) - Multi-Class Product Classification (Team A)/Datasets/shopping_queries_dataset_products.parquet')
df_p = shopping_data_p.to_pandas() #no sampling to match more shopping data
df_p = df_p.drop(columns=['product_locale']) #this is a duplicate from shopping_data

# merging
df_merged = pd.merge(df, df_p, on='product_id', how='inner')
print(f"number of products {len(pd.unique(df_merged['product_id']))}")
df_merged_no_null = df_merged.fillna("")
missing_values = df_merged_no_null.isnull().sum()
print(len(df_merged_no_null))

number of products 9963
10220


In [16]:
# concat dat
!pip install transformers
import torch
import transformers
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModel
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 88.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.1 MB/s eta 0:00:00


In [17]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [19]:
# Add this code before tokenization
def remove_stopwords(text):
    stop_words_english = set(stopwords.words('english'))
    stop_words_spanish = set(stopwords.words('spanish'))
    # stop_words_japanese = set(stopwords.words('japanese')) <-- resulted in error
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words_english
                      and word.lower() not in stop_words_spanish]
    return ' '.join(filtered_words)

In [73]:
# Get a random sample of 10000 elems in the merged dataset with null values replaced
sample_size = 10000

# Initialize an empty list to store the modified input sequences
input_sequences = []

# Apply stopword removal to each element in the 'input_sequences' list using a for loop
for text in df_merged_no_null.apply(lambda x: f"[CLS] {x['query']} [SEP] {x['product_title']} [SEP] {x['product_description']} [SEP] {x['product_bullet_point']} [SEP] {x['product_color']} [SEP] {x['product_bullet_point']} [SEP]", axis=1).tolist():
# for text in df_merged_no_null.apply(lambda x: f"[CLS] {x['product_title']} [SEP] {x['product_description']} [SEP] {x['product_bullet_point']} [SEP]", axis=1).tolist():
    cleaned_text = remove_stopwords(text)
    input_sequences.append(cleaned_text)
print(input_sequences[0], len(input_sequences))

[CLS] 100% cotton long sleeve shirt women [SEP] Anvil Ladies' Lightweight Long-Sleeve T-Shirt, Green Apple, Medium [SEP] 4.5 oz., preshrunk 100% combed ringspun cotton; 30 singles; Heathers-60/40 poly/combed ringspun cotton; Heather Grey - 90/10 combed ringspun cotton/poly; shoulder-to-shoulder tape; double-needle bottom hem; double-needle sleeve; Oeko-Tex Standard 100 Certified; features TearAway label; missy semi-fitted contoured silhouette sideseams; seamed collar [SEP] 4.5 oz., preshrunk 100% combed ringspun cotton 30 singles Heathers-60/40 poly/combed ringspun cotton Heather Grey - 90/10 combed ringspun cotton/poly shoulder-to-shoulder tape [SEP] Green Apple [SEP] 4.5 oz., preshrunk 100% combed ringspun cotton 30 singles Heathers-60/40 poly/combed ringspun cotton Heather Grey - 90/10 combed ringspun cotton/poly shoulder-to-shoulder tape [SEP] 10220


In [74]:
# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Example text to tokenize
# text = "This is an example sentence."

# Tokenize the text
tokens = tokenizer(input_sequences, padding=True, truncation=True, return_tensors="pt")

# Convert tokenized text to PyTorch tensors
input_ids = tokens['input_ids']
attention_mask = tokens['attention_mask']

In [75]:
from torch.utils.data import DataLoader, TensorDataset
def get_data_loader(features, max_seq_length, batch_size, shuffle=True):

    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
    all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.long)
    data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)

    dataloader = DataLoader(data, shuffle=shuffle, batch_size=batch_size)
    return dataloader
class BertInputItem(object):
  """An item with all the necessary attributes for finetuning BERT."""

  def __init__(self, text, input_ids, input_mask, segment_ids, label_id):
      self.text = text
      self.input_ids = input_ids
      self.input_mask = input_mask
      self.segment_ids = segment_ids
      self.label_id = label_id


In [51]:
def convert_examples_to_inputs(example_texts, example_labels, label2idx, max_seq_length, tokenizer, verbose=0):
      """Loads a data file into a list of `InputBatch`s."""

      input_items = []
      examples = zip(example_texts, example_labels)
      for (ex_index, (text, label)) in enumerate(examples):

          # Create a list of token ids
          input_ids = tokenizer.encode(f"[CLS] {text} [SEP]")
          if len(input_ids) > max_seq_length:
              input_ids = input_ids[:max_seq_length]

          # All our tokens are in the first input segment (id 0).
          segment_ids = [0] * len(input_ids)

          # The mask has 1 for real tokens and 0 for padding tokens. Only real
          # tokens are attended to.
          input_mask = [1] * len(input_ids)

          # Zero-pad up to the sequence length.
          padding = [0] * (max_seq_length - len(input_ids))
          input_ids += padding
          input_mask += padding
          segment_ids += padding

          assert len(input_ids) == max_seq_length
          assert len(input_mask) == max_seq_length
          assert len(segment_ids) == max_seq_length

          label_id = label2idx[label]

          input_items.append(
              BertInputItem(text=text,
                            input_ids=input_ids,
                            input_mask=input_mask,
                            segment_ids=segment_ids,
                            label_id=label_id))


      return input_items

In [76]:
rest_texts, test_texts, rest_labels, test_labels = train_test_split(input_sequences, df_merged_no_null['esci_label'].tolist(), test_size=0.1, random_state=1)
train_texts, dev_texts, train_labels, dev_labels = train_test_split(rest_texts, rest_labels, test_size=0.1, random_state=1)

target_names = list(set(df_merged_no_null['esci_label'].tolist()))
label2idx = {label: idx for idx, label in enumerate(target_names)}
print(label2idx)

{'C': 0, 'S': 1, 'E': 2, 'I': 3}


In [77]:
# inputItem = BertInputItem()
MAX_SEQ_LENGTH = 512
train_features = convert_examples_to_inputs(train_texts, train_labels, label2idx, MAX_SEQ_LENGTH, tokenizer, verbose=0)
dev_features = convert_examples_to_inputs(dev_texts, dev_labels, label2idx, MAX_SEQ_LENGTH, tokenizer)
test_features = convert_examples_to_inputs(test_texts, test_labels, label2idx, MAX_SEQ_LENGTH, tokenizer)

BATCH_SIZE = 16

train_dataloader = get_data_loader(train_features, MAX_SEQ_LENGTH, BATCH_SIZE, shuffle=True)
dev_dataloader = get_data_loader(dev_features, MAX_SEQ_LENGTH, BATCH_SIZE, shuffle=False)
test_dataloader = get_data_loader(test_features, MAX_SEQ_LENGTH, BATCH_SIZE, shuffle=False)

Token indices sequence length is longer than the specified maximum sequence length for this model (808 > 512). Running this sequence through the model will result in indexing errors


In [79]:
df_merged_no_null.shape

(10220, 14)

In [ ]:
# creating custom model with numerical features
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased")
additional_features = df_merged_no_null[["large_version", "small_version"]] #should we include this in the model?

#work in progress, need to create base model
import torch.optim as optim
from torch import nn

EPOCHS = 1

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(EPOCHS):
    model.train()
    for batch in train_dataloader:
        text_inputs, numerical_inputs, labels = batch

        # Clear gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(text_inputs, numerical_inputs)

        # Calculate loss
        loss = criterion(outputs, labels)

        # Backpropagation
        loss.backward()
        optimizer.step()
